#### Chunking with on device model to decide, for each processed document, which version of it is more coherent, human-readable

In [1]:
## 1.use local 3B model to determine which version is more human readable produce a cvs for result gathering and score calculation
## 2.use both GPT3 and 3B model to start chunking the selected version 
# if there is only normal read version, use that


In [ ]:
pdf_extracted_folder = "./../../Data/Extracted/Pdf"
receipts_extracted_folder = "./../../Data/Extracted/Receipts"
reports_extracted_folder = "./../../Data/Extracted/Reports"
with_format_extracted_folder = "./../../Data/Extracted/WithFormat"

pdf_input_folder = "./../../Data/Input/Pdf"
receipts_input_folder = "./../../Data/Input/Receipts"
reports_input_folder = "./../../Data/Input/Reports"
with_format_input_folder = "./../../Data/Input/WithFormat"

folder_matches = [(receipts_extracted_folder, receipts_input_folder), (pdf_extracted_folder, pdf_input_folder), (with_format_extracted_folder, with_format_input_folder), (reports_extracted_folder, reports_input_folder)]



In [2]:
import ollama
from pathlib import Path
import requests

class Ollama_asker:
    def __init__(self, url, model):
        self.url = url
        self.model = model
    def chunk_text_with_overlap(self, txt_file, chunk_size=500, overlap=100):
        """Splits text into overlapping chunks."""
        with open(txt_file, 'r', encoding='utf-8') as f:
            text = f.read()
        chunks = []
        start = 0
        while start < len(text):
            end_ptr = min(start + chunk_size, len(text))
            chunks.append(text[start:end_ptr])
            start += chunk_size - overlap  # Move forward while keeping overlap

        # for chunk in chunks:
        #     print(chunk)
        return chunks

    def rate(self,chunk):
        result = self.ask_ollama_from_txts(chunk, prompt ="Without Comments, reply only a number: rate the readability and usefullness of the text from 1(Bad) to 10(Good):")
        return result
    def chunk_summarize(self,chunk):
        result = self.ask_ollama_from_txts(chunk, prompt =
                                           '''Summarize the following text without adding any commentary, explanations, or disclaimers. Only return a clean, factual summary. If the content includes sensitive personal data, return an empty response.
                                            \nText:
                                            \n<insert text>''')
        return result

    def ask_ollama_from_txts(self, chunk, prompt=""):
            #return "" 
            # Send the text as context to Ollama
                response = ollama.chat(
                    model= self.model,
                    messages=[
                        {"role": "system", "content": "You are a helpful factual text-edit assitant."},
                        {"role": "user", "content": f"{prompt}:\n{chunk}"}
                    ]
                )
                return response["message"]["content"]
           




#### For each extracted file, get its average chunk socres, original chunk, and chunk summary to one row

In [146]:
import numpy as np
import pandas as pd
import csv 
import json
import os
url = "http://127.0.0.1:11434"
asker = Ollama_asker("", "llama3.2:3b")
data = []
separator = "\n}<->{\n"
score_base = "/Users/seanhuang/Grad/adapter/Data/Input/"

def score_category(cat: int, json_path: str):
    header_written = Path(json_path).exists()
    extracted_folder, input_folder = folder_matches[cat]
    print(f"Processing {extracted_folder} to {input_folder}")
    extracted_folder = Path(extracted_folder)
    input_folder = Path(input_folder)
    file = open(json_path, mode="a", encoding="utf-8")

    try: 
        file.write("[\n")  
        for txt_file in extracted_folder.rglob("*.txt"):  # Get all .txt files in the folder
            print(f"checking file: {txt_file}")
            chunks = asker.chunk_text_with_overlap(txt_file=txt_file)


            # Get ratings
            ratings = [asker.rate(chunk=c) for c in chunks]
            summaries = [asker.chunk_summarize(chunk=c) for c in chunks]
            # # Convert ratings to strings
            # ratings = ratings.tolist() if isinstance(ratings, np.ndarray) else ratings
            # chunks = chunks.tolist() if isinstance(chunks, np.ndarray) else chunks
            # summaries = summaries.tolist() if isinstance(summaries, np.ndarray) else summaries

            # Join everything
            all_rating = separator.join(ratings)
            all_summary = separator.join(summaries)
            big_chunk = separator.join(chunks)

            json_entry = {
                "Scores": all_rating,
                "Chunks": big_chunk,
                "Summaries": all_summary
            }
            json.dump({str(txt_file) :json_entry}, file, indent=4)
            # Append data to CSV
            print(f"Saved: {str(txt_file)}")
        file.write("\n]") 
        file.close()
        print(f"Finished processing {extracted_folder} to {input_folder}")
    except Exception as e:
        file.write("\n]") 
        file.close()



# # Convert the list of dictionaries into a DataFrame
# df = pd.DataFrame(data)
# df.to_csv(csv_path, index=False, quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
#"Without Comments, summarize in original languange with useful information, below 100 or no words (if not able to comment nothing):"
score_category(0,json_path=score_base + "receipts_scores_senilent_if_sensitive.json")


Processing /Users/seanhuang/Grad/adapter/Data/Extracted/Receipts to /Users/seanhuang/Grad/adapter/Data/Input/Receipts
checking file: /Users/seanhuang/Grad/adapter/Data/Extracted/Receipts/2022/us/walmart_20221228_normal.txt
Saved: /Users/seanhuang/Grad/adapter/Data/Extracted/Receipts/2022/us/walmart_20221228_normal.txt
checking file: /Users/seanhuang/Grad/adapter/Data/Extracted/Receipts/2022/us/snacks_20221210_015_ocr.txt
Saved: /Users/seanhuang/Grad/adapter/Data/Extracted/Receipts/2022/us/snacks_20221210_015_ocr.txt
checking file: /Users/seanhuang/Grad/adapter/Data/Extracted/Receipts/2022/us/tap_20221210_011_ocr.txt
Saved: /Users/seanhuang/Grad/adapter/Data/Extracted/Receipts/2022/us/tap_20221210_011_ocr.txt
checking file: /Users/seanhuang/Grad/adapter/Data/Extracted/Receipts/2022/us/dennys_20221210_002_ocr.txt
Saved: /Users/seanhuang/Grad/adapter/Data/Extracted/Receipts/2022/us/dennys_20221210_002_ocr.txt
checking file: /Users/seanhuang/Grad/adapter/Data/Extracted/Receipts/2022/us/wal

KeyboardInterrupt: 

In [122]:
score_category(1)

Processing /Users/seanhuang/Grad/adapter/Data/Extracted/Pdf to /Users/seanhuang/Grad/adapter/Data/Input/Pdf
checking file: /Users/seanhuang/Grad/adapter/Data/Extracted/Pdf/aa932e9b0226461291e5f75cc35d2e13c0ab3218_normal.txt
Saved: /Users/seanhuang/Grad/adapter/Data/Extracted/Pdf/aa932e9b0226461291e5f75cc35d2e13c0ab3218_normal.txt
checking file: /Users/seanhuang/Grad/adapter/Data/Extracted/Pdf/KQKKOFZRA2MSM7S4EEGOTH2CPEJOUNWD_normal.txt
Saved: /Users/seanhuang/Grad/adapter/Data/Extracted/Pdf/KQKKOFZRA2MSM7S4EEGOTH2CPEJOUNWD_normal.txt
checking file: /Users/seanhuang/Grad/adapter/Data/Extracted/Pdf/A5PO2R5VRU5CSRP4JRS5SZNUTAD7P3HA_ocr.txt
Saved: /Users/seanhuang/Grad/adapter/Data/Extracted/Pdf/A5PO2R5VRU5CSRP4JRS5SZNUTAD7P3HA_ocr.txt
checking file: /Users/seanhuang/Grad/adapter/Data/Extracted/Pdf/AEDZUJF6HFEQUC3GZMVSC5EVDPBEJ7TF_normal.txt
Saved: /Users/seanhuang/Grad/adapter/Data/Extracted/Pdf/AEDZUJF6HFEQUC3GZMVSC5EVDPBEJ7TF_normal.txt
checking file: /Users/seanhuang/Grad/adapter/Data/

KeyboardInterrupt: 

In [ ]:
score_category(2)

Processing /Users/seanhuang/Grad/adapter/Data/Extracted/Reports to /Users/seanhuang/Grad/adapter/Data/Input/Reports
checking file: /Users/seanhuang/Grad/adapter/Data/Extracted/Reports/2009.14082v2_normal.txt


In [119]:
score_category(3)

Processing /Users/seanhuang/Grad/adapter/Data/Extracted/WithFormat to /Users/seanhuang/Grad/adapter/Data/Input/WithFormat
checking file: /Users/seanhuang/Grad/adapter/Data/Extracted/WithFormat/Company Documents Dataset/CompanyDocuments/invoices/invoice_10830_normal.txt
Saved: /Users/seanhuang/Grad/adapter/Data/Extracted/WithFormat/Company Documents Dataset/CompanyDocuments/invoices/invoice_10830_normal.txt
checking file: /Users/seanhuang/Grad/adapter/Data/Extracted/WithFormat/Company Documents Dataset/CompanyDocuments/invoices/invoice_10807_ocr.txt
Saved: /Users/seanhuang/Grad/adapter/Data/Extracted/WithFormat/Company Documents Dataset/CompanyDocuments/invoices/invoice_10807_ocr.txt
checking file: /Users/seanhuang/Grad/adapter/Data/Extracted/WithFormat/Company Documents Dataset/CompanyDocuments/invoices/invoice_10817_ocr.txt
Saved: /Users/seanhuang/Grad/adapter/Data/Extracted/WithFormat/Company Documents Dataset/CompanyDocuments/invoices/invoice_10817_ocr.txt
checking file: /Users/sean

KeyboardInterrupt: 

#### Collect all the Json results in in SummaryNScores, then see which version of the document is more coherent, human-readable based on scores

In [133]:
formatted_pdfs = "/Users/seanhuang/Grad/adapter/LLMAdapter/SummaryNScores/formatted_scores.json"
receipt_pdfs = "/Users/seanhuang/Grad/adapter/LLMAdapter/SummaryNScores/receipt_scores.json"
report_pdfs = "/Users/seanhuang/Grad/adapter/LLMAdapter/SummaryNScores/report_scores.json"
random_pdfs = "/Users/seanhuang/Grad/adapter/LLMAdapter/SummaryNScores/random_scores.json"

In [145]:
import json
import os
with open(formatted_pdfs, "r") as f1, open(receipt_pdfs, "r") as f2, open(report_pdfs, "r") as f3, open(random_pdfs, "r") as f4:
    formatted = json.load(f1)
    receipts = json.load(f2)
    reports = json.load(f3)
    randoms = json.load(f4)
    print("PDFs with formats",len(formatted))
    print("PDFs of recipts that need ocr", len(receipts))
    print("PDFs of well written long papers",len(reports))
    print("PDFs of all sorts",len(randoms))
    formatted = sorted(formatted, key=lambda x: list(x.keys())[0])
    print(formatted[0])
    print(formatted[1])
    print(formatted[2])
    print(formatted[3])
    print(formatted[4])
    print(formatted[5])
    receipts = sorted(receipts, key=lambda x: list(x.keys())[0])
    print(receipts[0])
    print(receipts[1])
    print(receipts[2])
    print(receipts[3])
    print(receipts[4])
    print(receipts[5])
    reports = sorted(reports, key=lambda x: list(x.keys())[0])
    print(reports[0])
    print(reports[1])
    print(reports[2])
    print(reports[3])
    print(reports[4])
    print(reports[5])
    randoms = sorted(randoms, key=lambda x: list(x.keys())[0])
    print(randoms[0])
    print(randoms[1])
    print(randoms[2])
    print(randoms[3])
    print(randoms[4])
    print(randoms[5])

    


PDFs with formats 4840
PDFs of recipts that need ocr 2441
PDFs of well written long papers 98
PDFs of all sorts 600
{'/home/yxuhuang/adapter/Data/Extracted/WithFormat/Company Documents Dataset/CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_1_normal.txt': {'Scores': '8', 'Chunks': 'Stock Report for 2016-07 \n Category : Beverages  \n id category : 1  \nProduct\nUnits Sold\nUnits in Stock\nUnit Price\nChang\n105\n17\n19\nGuaraná Fantástica\n43\n20\n4.5\nSteeleye Stout\n20\n20\n18\nChartreuse verte\n48\n69\n18\nOutback Lager\n41\n15\n15\nLakkalikööri\n15\n57\n18\n\n--------------------------------------------------\n', 'Summaries': 'Stock Report for 2016-07 \n id category : 1 \nBeverages: \n- Guaraná Fantástica: 43 units sold, 20 in stock, $4.5/unit \n- Steeleye Stout: 20 units sold, 20 in stock, $18/unit \n- Chartreuse verte: 48 units sold, 69 in stock, $18/unit \n- Outback Lager: 41 units sold, 15 in stock, $15/unit \n- Lakkalikööri: 15 units sol